In [2]:
import pandas as pd

### Summary statistics

In [3]:
sales = pd.read_csv("sales_subset.csv")
sales.head()

,Unnamed: 0,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,0,1,A,1,2010-02-05,24924.50,False,5.727778,0.679451,8.106
1,1,1,A,1,2010-03-05,21827.90,False,8.055556,0.693452,8.106
2,2,1,A,1,2010-04-02,57258.43,False,16.816667,0.718284,7.808
3,3,1,A,1,2010-05-07,17413.94,False,22.527778,0.748928,7.808
4,4,1,A,1,2010-06-04,17558.09,False,27.050000,0.714586,7.808


In [4]:
print(sales["weekly_sales"].mean())
print(sales["weekly_sales"].median())

23843.95014850566
12049.064999999999


In [5]:
# Print the maximum of the date column
print(sales["date"].max())

# Print the minimum of the date column
print(sales["date"].min())

2012-10-26
2010-02-05


#### Efficient summary

In [6]:
# A custom IQR function
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

# Print IQR of the temperature_c column
print(sales['temperature_c'].agg(iqr))

16.583333333333336


In [7]:
# A custom IQR function
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

# Update to print IQR of temperature_c, fuel_price_usd_per_l, & unemployment
print(sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg(iqr))

temperature_c           16.583333
fuel_price_usd_per_l     0.073176
unemployment             0.565000
dtype: float64


In [8]:
# Import NumPy and create custom IQR function
import numpy as np
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

# Update to print IQR and median of temperature_c, fuel_price_usd_per_l, & unemployment
print(sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr,np.median]))

        temperature_c  fuel_price_usd_per_l  unemployment
iqr         16.583333              0.073176         0.565
median      16.966667              0.743381         8.099


/tmp/ipykernel_2716/2489884941.py:7: FutureWarning: The provided callable <function median at 0x7f9d90299c60> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  print(sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr,np.median]))
/tmp/ipykernel_2716/2489884941.py:7: FutureWarning: The provided callable <function median at 0x7f9d90299c60> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  print(sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr,np.median]))
/tmp/ipykernel_2716/2489884941.py:7: FutureWarning: The provided callable <function median at 0x7f9d90299c60> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass th

#### Cumulative statistics

In [9]:
# Sort sales_1_1 by date
sales_1_1 = sales.sort_values("date")

# Get the cumulative sum of weekly_sales, add as cum_weekly_sales col
sales_1_1["cum_weekly_sales"] = sales_1_1["weekly_sales"].cumsum()

# Get the cumulative max of weekly_sales, add as cum_max_sales col
sales_1_1["cum_max_sales"] = sales_1_1["weekly_sales"].cummax()

# See the columns you calculated
print(sales_1_1[["date", "weekly_sales", "cum_weekly_sales", "cum_max_sales"]])

             date  weekly_sales  cum_weekly_sales  cum_max_sales
0      2010-02-05      24924.50      2.492450e+04       24924.50
6437   2010-02-05      38597.52      6.352202e+04       38597.52
1249   2010-02-05       3840.21      6.736223e+04       38597.52
6449   2010-02-05      17590.59      8.495282e+04       38597.52
6461   2010-02-05       4929.87      8.988269e+04       38597.52
...           ...           ...               ...            ...
3592   2012-10-05        440.00      2.568932e+08      293966.05
8108   2012-10-05        660.00      2.568938e+08      293966.05
10773  2012-10-05        915.00      2.568947e+08      293966.05
6257   2012-10-12          3.00      2.568947e+08      293966.05
3384   2012-10-26        -21.63      2.568947e+08      293966.05

[10774 rows x 4 columns]


### Aggregating dataframe

In [10]:
store_types = sales.drop_duplicates(subset=["store", "type"]) #去除store和type同时重复的行

store_depts = sales.drop_duplicates(subset=["store", "department"])

holiday_dates = sales[sales["is_holiday"]==True].drop_duplicates(subset="date")


#### Counting categorical variables


In [11]:
store_counts = store_types["type"].value_counts()
print(store_counts)

type
A    11
B     1
Name: count, dtype: int64


In [12]:
store_props = store_types["type"].value_counts(normalize=True)
print(store_props)

type
A    0.916667
B    0.083333
Name: proportion, dtype: float64


In [13]:
dept_counts_sorted = store_depts["department"].value_counts()
print(dept_counts_sorted)

department
1     12
2     12
3     12
4     12
5     12
      ..
37    10
48     8
50     6
39     4
43     2
Name: count, Length: 80, dtype: int64


In [14]:
# Get the proportion of stores in each department and sort
dept_props_sorted = store_depts["department"].value_counts(sort=True, normalize=True)
print(dept_props_sorted)

department
1     0.012917
2     0.012917
3     0.012917
4     0.012917
5     0.012917
        ...   
37    0.010764
48    0.008611
50    0.006459
39    0.004306
43    0.002153
Name: proportion, Length: 80, dtype: float64


#### Grouped summary statistics
What percentage of sales in each store

In [15]:
# Calc total weekly sales
sales_all = sales["weekly_sales"].sum()
sales_all

# Subset for type A stores, calc total weekly sales
sales_A = sales[sales["type"] == "A"]["weekly_sales"].sum()

# Subset for type B stores, calc total weekly sales
sales_B = sales[sales["type"] == "B"]["weekly_sales"].sum()

# Subset for type C stores, calc total weekly sales
sales_C = sales[sales["type"] == "C"]["weekly_sales"].sum()
# Get proportion for each type
sales_propn_by_type = [sales_A, sales_B, sales_C] / sales_all
print(sales_propn_by_type)

[0.9097747 0.0902253 0.       ]


With groupby

In [16]:
# Group by type; calc total weekly sales
sales_by_type = sales.groupby("type")["weekly_sales"].sum()

# Get proportion for each type
sales_propn_by_type = sales_by_type / sales_by_type.sum()
print(sales_propn_by_type)

type
A    0.909775
B    0.090225
Name: weekly_sales, dtype: float64


In [ ]:
# From previous step
sales_by_type = sales.groupby("type")["weekly_sales"].sum()

# Group by type and is_holiday; calc total weekly sales
sales_by_type_is_holiday = sales.groupby(["type", "is_holiday"])["weekly_sales"].sum() #如果是2个category就看能出多少个组合
print(sales_by_type_is_holiday)

type  is_holiday
A     False         2.336927e+08
      True          2.360181e+04
B     False         2.317678e+07
      True          1.621410e+03
Name: weekly_sales, dtype: float64


In [18]:
# Import numpy with the alias np
import numpy as np

# For each store type, aggregate weekly_sales: get min, max, mean, and median
sales_stats = sales.groupby("type")["weekly_sales"].agg([min,max,np.mean, np.median])

# Print sales_stats
sales_stats

/tmp/ipykernel_2716/2192151368.py:5: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  sales_stats = sales.groupby("type")["weekly_sales"].agg([min,max,np.mean, np.median])
/tmp/ipykernel_2716/2192151368.py:5: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  sales_stats = sales.groupby("type")["weekly_sales"].agg([min,max,np.mean, np.median])
/tmp/ipykernel_2716/2192151368.py:5: FutureWarning: The provided callable <function mean at 0x7f9d9038afc0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  sales_

,min,max,mean,median
type,,,,
A,-1098.0,293966.05,23674.667242,11943.92
B,-798.0,232558.51,25696.678370,13336.08


In [19]:


# For each store type, aggregate unemployment and fuel_price_usd_per_l: get min, max, mean, and median
unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([min,max,np.mean,np.median])

# Print unemp_fuel_stats
unemp_fuel_stats

/tmp/ipykernel_2716/3537935694.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([min,max,np.mean,np.median])
/tmp/ipykernel_2716/3537935694.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([min,max,np.mean,np.median])
/tmp/ipykernel_2716/3537935694.py:2: FutureWarning: The provided callable <function mean at 0x7f9d9038afc0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To ke

unemployment                         fuel_price_usd_per_l            \
              min    max      mean median                  min       max   
type                                                                       
A           3.879  8.992  7.972611  8.067             0.664129  1.107410   
B           7.170  9.765  9.279323  9.199             0.760023  1.107674   

                          
          mean    median  
type                      
A     0.744619  0.735455  
B     0.805858  0.803348

#### pivot table

In [20]:
sales.groupby("type")["weekly_sales"].mean()

type
A    23674.667242
B    25696.678370
Name: weekly_sales, dtype: float64

In [ ]:
sales.pivot_table(values="weekly_sales", index="type") #default is mean

,weekly_sales
type,
A,23674.667242
B,25696.678370


In [22]:
sales.pivot_table(values="weekly_sales", index="type", aggfunc=np.median)

/tmp/ipykernel_2716/1217410532.py:1: FutureWarning: The provided callable <function median at 0x7f9d90299c60> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sales.pivot_table(values="weekly_sales", index="type", aggfunc=np.median)


,weekly_sales
type,
A,11943.92
B,13336.08


In [23]:
sales.pivot_table(values="weekly_sales", index="type", aggfunc=[np.mean,np.median])

/tmp/ipykernel_2716/267361341.py:1: FutureWarning: The provided callable <function mean at 0x7f9d9038afc0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  sales.pivot_table(values="weekly_sales", index="type", aggfunc=[np.mean,np.median])
/tmp/ipykernel_2716/267361341.py:1: FutureWarning: The provided callable <function median at 0x7f9d90299c60> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sales.pivot_table(values="weekly_sales", index="type", aggfunc=[np.mean,np.median])


,mean,median
,weekly_sales,weekly_sales
type,,
A,23674.667242,11943.92
B,25696.678370,13336.08


Pivot on two variables

In [ ]:
sales.groupby(["type", "is_holiday"])["weekly_sales"].mean()

/tmp/ipykernel_2716/2960606047.py:1: FutureWarning: The provided callable <function mean at 0x7f9d9038afc0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  sales.groupby(["type", "is_holiday"])["weekly_sales"].agg(np.mean)


type  is_holiday
A     False         23768.583523
      True            590.045250
B     False         25751.980533
      True            810.705000
Name: weekly_sales, dtype: float64

In [26]:
sales.pivot_table(values="weekly_sales", index="type", columns="is_holiday")

is_holiday,False,True
type,,
A,23768.583523,590.04525
B,25751.980533,810.70500


In [27]:
sales.pivot_table(values="weekly_sales", index="type", columns="is_holiday", fill_value=0) # fill nan with 0

is_holiday,False,True
type,,
A,23768.583523,590.04525
B,25751.980533,810.70500


In [28]:
sales.pivot_table(values="weekly_sales", index="type", columns="is_holiday", margins=True)

is_holiday,False,True,All
type,,,
A,23768.583523,590.045250,23674.667242
B,25751.980533,810.705000,25696.678370
All,23934.913873,600.552857,23843.950149
